In [1]:
from datasets import load_dataset
from utils.data_preprocessing import preprocess_data

HF_TOKEN = dotenv_values(".env.base")['HF_TOKEN']
# If the dataset is gated/private, make sure you have run huggingface-cli login
dataset = load_dataset("ferrazzipietro/e3c-sentences", token=HF_TOKEN)

/Users/pietroferrazzi/Desktop/dottorato/mistral_finetuning/.env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
from config import preprocessing_params
from utils.data_preprocessor import DataPreprocessor
import pandas as pd
import os
def preprocess_data(hf_dataset):
    data_preprocessor = DataPreprocessor()
    splits = ['en.layer1', 'en.layer2', 'en.layer2.validation', 'en.layer3',
            'es.layer1', 'es.layer2', 'es.layer2.validation', 'es.layer3',
            'eu.layer1', 'eu.layer2', 'eu.layer2.validation', 'eu.layer3',
            'it.layer1', 'it.layer2', 'it.layer2.validation', 'it.layer3',
            'fr.layer1', 'fr.layer2', 'fr.layer2.validation', 'fr.layer3']
    for split_name in splits:
        hf_dataset[split_name] = data_preprocessor.apply(data=hf_dataset[split_name], 
                                                        task=preprocessing_params.task, 
                                                        instruction_on_response_format=preprocessing_params.instruction_on_response_format, 
                                                        n_shots=preprocessing_params.n_shots, 
                                                        offset=preprocessing_params.offset, 
                                                        tokenizer=preprocessing_params.tokenizer, 
                                                        list_of_examples=preprocessing_params.list_of_examples,
                                                        list_of_responses=preprocessing_params.list_of_responses)
    return hf_dataset

def split_layer_into_train_test_(hf_dataset, split_name='en.layer1'):
    mapping = {'en.layer1': 'train_labels_en.txt', 
               'en.layer2': 'train_labels_en.txt',
               'en.layer2.validation': 'test_labels_en.txt',
               'en.layer3': 'test_labels_en.txt',
               'es.layer1': 'train_labels_es.txt',
               'es.layer2': 'train_labels_es.txt',
               'es.layer2.validation': 'test_labels_es.txt',
               'es.layer3': 'test_labels_es.txt',
               'eu.layer1': 'train_labels_eu.txt',
               'eu.layer2': 'train_labels_eu.txt',
               'eu.layer2.validation': 'test_labels_eu.txt',
               'eu.layer3': 'test_labels_eu.txt',
               'it.layer1': 'train_labels_it.txt',
               'it.layer2': 'train_labels_it.txt',
               'it.layer2.validation': 'test_labels_it.txt',
               'it.layer3': 'test_labels_it.txt'}
    labels_path = mapping[split_name]
    with open(os.path.join('data', labels_path), 'r') as file:
        file_content = file.read()
    labels = file_content.split(", ")
    labels = [label[1:-1] for label in labels]
    return labels
    

In [28]:
labels = split_layer_into_train_test_(dataset, split_name='en.layer1')

print(len(labels))
len(set(dataset['en.layer1']['original_id']))

36


84